# Demo "Relaxed Releases with DB-Workloadtests": Workload Replay

## Disclaimer
THIS CODE AND INFORMATION ARE PROVIDED "AS IS" WITHOUT WARRANTY OF ANY KIND, EITHER EXPRESSED OR IMPLIED, INCLUDING BUT NOT LIMITED
TO THE IMPLIED WARRANTIES OF MERCHANTABILITY AND/OR FITNESS FOR
PARTICULAR PURPOSE. 

CODE IS WRITTEN FOR DEMO PURPOSES 

(C) Martin Guth and others, 2020

## Prerequisites
* SQL Server 2008 or newer (tested with SQL Server 2019 on Linux resp Docker)
* [StackOverflow-Database](https://www.brentozar.com/archive/2015/10/how-to-download-the-stack-overflow-database-via-bittorrent/) (tested with StackOverflow2010 aka small edition)
* [sp_whoIsActive](http://whoisactive.com/downloads/) to monitor activity in the database
* [WorkloadTools](https://github.com/spaghettidba/WorkloadTools/releases) to capture the workload



## Preparations
### Restore
First we bring the database to the state we need to run the workload (simplified here because no activity on the test system and therfore one full backup restore suffices.)
Hint: You might want to change the path to the backupfile (especially if you are running SQL Server on Windows).

In [6]:
ALTER DATABASE [StackOverflow2010] SET SINGLE_USER WITH ROLLBACK IMMEDIATE;
RESTORE DATABASE [StackOverflow2010] 
    FROM DISK = N'/var/opt/mssql/backup/StackOverflow2010.bak' 
    WITH REPLACE, STATS=10, RECOVERY;
ALTER DATABASE [StackOverflow2010] SET MULTI_USER;

10 percent processed.

20 percent processed.

30 percent processed.

40 percent processed.

50 percent processed.

...

100 percent processed.

Processed 1096528 pages for database 'StackOverflow2010', file 'StackOverflow2010' on file 1.

Processed 2 pages for database 'StackOverflow2010', file 'StackOverflow2010_log' on file 1.

RESTORE DATABASE successfully processed 1096530 pages in 16.209 seconds (528.511 MB/sec).

Total execution time: 00:00:18.460

### Make sure Query Store is on and configured appropriately (needs SQL Server 2016+)
Caution: Configuration <mark> just for demo purposes</mark> 
* Capture all queries (QUERY_CAPTURE_MODE ALL)
* Write to disk and aggregate WaitStats every 5 minutes

For a bigger workload different query store settings might be more appropriate!

In [7]:
USE [master]
GO
ALTER DATABASE [StackOverflow2010] SET QUERY_STORE = ON
GO
ALTER DATABASE [StackOverflow2010] SET QUERY_STORE (OPERATION_MODE = READ_WRITE, DATA_FLUSH_INTERVAL_SECONDS = 300, INTERVAL_LENGTH_MINUTES = 5, MAX_STORAGE_SIZE_MB = 2048, QUERY_CAPTURE_MODE = ALL)
GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.174

### Create a database for Analysis-Results
WorkloadTools offers the "AnalysisConsumer" to save WorkloadAnalyses to a database.
It is recommended to store results of different workloads in different schemas. The tables where analysis results are logged always have the same names.

Here we create a new database "WorkloadAnalysis" with a schema "baseline":


In [8]:
USE master;
IF EXISTS(SELECT * FROM sys.databases WHERE name = 'WorkloadAnalysis')
BEGIN
	ALTER DATABASE WorkloadAnalysis SET SINGLE_USER WITH ROLLBACK IMMEDIATE;
	DROP DATABASE WorkloadAnalysis ;
END

CREATE DATABASE WorkloadAnalysis;
GO
USE WorkloadAnalysis
GO
CREATE SCHEMA [baseline];
GO

/* Plausibility Checks */
SELECT name FROM sys.databases WHERE name = 'WorkloadAnalysis';
SELECT name FROM WorkloadAnalysis.sys.schemas WHERE name = 'baseline';
SELECT COUNT(*) AS nrOfTables FROM WorkloadAnalysis.sys.tables;

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:01.485

name WorkloadAnalysis

name baseline

anzTabellen 0

##  Configuration Workload Replay

The configuration is again done via a JSON file. Take a look at the file Replay.json. It has the following content:

```
{
    "Controller": {

        "Listener":
        {
            "__type": "FileWorkloadListener",
            "Source": "C:\\SQL\\workloadtests\\Demo\\StackOverflow_2010.sqlite",
            "SynchronizationMode": "true"
        },

        "Consumers":
        [
            {
                "__type": "AnalysisConsumer",
                "ConnectionInfo":
            		{
            			"ServerName": ".",
            			"DatabaseName": "WorkloadAnalysis",
            			"SchemaName": "baseline"
            		},
            		"UploadIntervalSeconds": 30
            },
      	    {
                "__type": "ReplayConsumer",
                "ConnectionInfo":
            		{
            			"ServerName": ".",
            			"DatabaseName": "StackOverflow2010"
            		},
            		"DisplayWorkerStats": "true",
            		"ConsumeResults": "false",
            		"WorkerStatsCommandCount": 1000,
            		"MimicApplicationName": "true",
                	"QueryTimeoutSeconds": 240
            }
        ]
    }
}
```
- Workload is read from the file "StackOverflow_2010.sqlite"
- Replay works synchronous that means pauses in the workload are respected. With an asynchronous replay("SynchronizationMode": "false") all statements would be run as fast as possible.
- Analysis-results are saved to the database "WorkloadAnalysis" and schema "baseline". Results are written every 30 seconds.
- Replay runs on the database "StackOverflow2010". That means all recorded activity without a database name in it will be run by default in that database.

## Start the Replay
Replaying the workload can be done with this simple command in Powershell
```
& "C:\Program Files\WorkloadTools\SqlWorkload.exe" --file "C:\SQL\workloadtests\Demo\Replay.json" --Log "C:\SQL\workloadtests\Demo\replay$(Get-Date -Format "yyyyMMdd_HHmmss").log"

```

## Clone the Database
We are now finished with the replay. Query Store has recorded a bunch of statistics during the first run of the replay (aka baseline).
We would like to save these statistics for later comparision. A comfortable solution for that is the command DBCC CLONEDATABASE. The clone contains just metadata (table definition, statistics AND Query Store :-)).
The command itself exists starting from SQL Server 2014 SP3...regarding Query Store it makes sense starting from SQL Server 2016 (because Query Store was introduced with that release).


In [9]:
DBCC CLONEDATABASE (StackOverflow2010, StackOverflow2010_baseline);    
GO 

Database cloning for 'StackOverflow2010' has started with target as 'StackOverflow2010_baseline'.

Database cloning for 'StackOverflow2010' has finished. Cloned database is 'StackOverflow2010_baseline'.

Database 'StackOverflow2010_baseline' is a cloned database. This database should be used for diagnostic purposes only and is not supported for use in a production environment.

DBCC execution completed. If DBCC printed error messages, contact your system administrator.

Total execution time: 00:00:03.494

In [10]:
-- test if QueryStoreData is available
SELECT  TOP 10 * FROM  StackOverflow2010_baseline.sys.query_store_query

(10 rows affected)

Total execution time: 00:00:00.143

query_id query_text_id context_settings_id object_id batch_sql_handle query_hash is_internal_query query_parameterization_type query_parameterization_type_desc initial_compile_start_time last_compile_start_time last_execution_time last_compile_batch_sql_handle last_compile_batch_offset_start last_compile_batch_offset_end count_compiles avg_compile_duration last_compile_duration avg_bind_duration last_bind_duration avg_bind_cpu_time last_bind_cpu_time avg_optimize_duration last_optimize_duration avg_optimize_cpu_time last_optimize_cpu_time avg_compile_memory_kb last_compile_memory_kb max_compile_memory_kb is_clouddb_internal_query 1 1 2 0 NULL 0x9AF5F37C001ADA50 1 0 None 2020-05-11 11:34:11.4130000 +00:00 2020-05-11 11:34:11.4130000 +00:00 2020-05-11 11:34:11.5570000 +00:00 0x020000004F042005CCA12936BB9A989A5EF9B4B0A14192A40000000000000000000000000000000000000000 0 678 1 15647 15647 14810 14810 11762 11762 837 837 837 837 208 208 208 0 2 2 1 901578250 NULL 0xDE6AEB2BA83F2BD2 0 0 None 2020-05-11 11:34:11.3900000 +00:00 2020-05-11 11:34:11.3900000 +00:00 2020-05-11 12:04:25.7670000 +00:00 0x03000A000AFEBC35325D9800B5AB000001000000000000000000000000000000000000000000000000000000 852 1028 1 215380 215380 23290 23290 15843 15843 192090 192090 150494 150494 288 288 288 0 3 3 1 901578250 NULL 0xFE7A3B6DA3924B7A 0 0 None 2020-05-11 11:34:11.6070000 +00:00 2020-05-11 11:34:11.6070000 +00:00 2020-05-11 12:04:25.7670000 +00:00 0x03000A000AFEBC35325D9800B5AB000001000000000000000000000000000000000000000000000000000000 1042 1978 1 987 987 266 266 266 266 721 721 721 721 216 216 216 0 4 4 2 0 NULL 0xEC224314160D73F5 1 0 None 2020-05-11 11:34:13.1670000 +00:00 2020-05-11 11:34:13.1670000 +00:00 2020-05-11 11:34:13.6970000 +00:00 0x0200000049906C25F845B9E57DDCBB6AA5E2434590D1E21B0000000000000000000000000000000000000000 0 678 1 1238 1238 210 210 210 210 1028 1028 1028 1028 200 200 200 0 5 5 1 949578421 NULL 0xDAB9585A0BB813F8 0 0 None 2020-05-11 11:34:13.1630000 +00:00 2020-05-11 11:34:13.1630000 +00:00 2020-05-11 12:03:29.2970000 +00:00 0x03000A00B56A9938465D9800B5AB000001000000000000000000000000000000000000000000000000000000 736 912 1 541418 541418 593 593 593 593 540825 540825 452924 452924 336 336 336 0 6 6 1 949578421 NULL 0x439BDBF6C1B5E939 0 0 None 2020-05-11 11:34:13.7070000 +00:00 2020-05-11 11:34:13.7070000 +00:00 2020-05-11 12:03:29.2970000 +00:00 0x03000A00B56A9938465D9800B5AB000001000000000000000000000000000000000000000000000000000000 1374 2594 1 998 998 325 325 325 325 673 673 673 673 200 200 200 0 7 7 1 933578364 NULL 0x8FF146B200807527 0 0 None 2020-05-11 11:34:13.9470000 +00:00 2020-05-11 11:34:13.9470000 +00:00 2020-05-11 12:06:10.7770000 +00:00 0x03000A007C46A5373F5D9800B5AB000001000000000000000000000000000000000000000000000000000000 94 246 1 2285 2285 1100 1100 1100 1100 1185 1185 1185 1185 240 240 240 0 8 8 2 0 NULL 0x6603D1199152BD76 1 0 None 2020-05-11 11:34:33.6630000 +00:00 2020-05-11 11:34:33.6630000 +00:00 2020-05-11 11:34:35.7130000 +00:00 0x02000000E7FC27213095D920B37EC4357FE7426B5C6E84940000000000000000000000000000000000000000 0 678 1 1226 1226 213 213 213 213 1013 1013 1013 1013 208 208 208 0 9 9 1 917578307 NULL 0x738C2958B0C85382 0 0 None 2020-05-11 11:34:33.6630000 +00:00 2020-05-11 11:34:33.6630000 +00:00 2020-05-11 12:02:16.7300000 +00:00 0x03000A004322B136385D9800B5AB000001000000000000000000000000000000000000000000000000000000 446 622 1 2058131 2058131 257 257 257 257 2057874 2057874 1267510 1267510 304 304 304 0 10 10 1 917578307 NULL 0xA1D4571BE443659F 0 0 None 2020-05-11 11:34:35.7200000 +00:00 2020-05-11 11:34:35.7200000 +00:00 2020-05-11 12:02:16.7300000 +00:00 0x03000A004322B136385D9800B5AB000001000000000000000000000000000000000000000000000000000000 636 2214 1 2011 2011 380 380 380 380 1631 1631 1631 1631 224 224 224 0

# Perparations for the second replay
First we reset the database by restoring the backup and resetting the QueryStore settings. This code is identical to the execution for the first replay (refer to this section of the notebook for further comments on that).
Then we create a schema "change" in the Database "WorkloadAnalysis" for storage of analyses regarding the second replay.


In [15]:
ALTER DATABASE [StackOverflow2010] SET SINGLE_USER WITH ROLLBACK IMMEDIATE;
RESTORE DATABASE [StackOverflow2010] 
    FROM DISK = N'/var/opt/mssql/backup/StackOverflow2010.bak' 
    WITH REPLACE, STATS=10, RECOVERY;
ALTER DATABASE [StackOverflow2010] SET MULTI_USER;

Nonqualified transactions are being rolled back. Estimated rollback completion: 0%.

Nonqualified transactions are being rolled back. Estimated rollback completion: 100%.

10 percent processed.

20 percent processed.

30 percent processed.

...

100 percent processed.

Processed 1096528 pages for database 'StackOverflow2010', file 'StackOverflow2010' on file 1.

Processed 2 pages for database 'StackOverflow2010', file 'StackOverflow2010_log' on file 1.

RESTORE DATABASE successfully processed 1096530 pages in 17.409 seconds (492.080 MB/sec).

Total execution time: 00:00:22.518

In [16]:
USE [master]
GO
ALTER DATABASE [StackOverflow2010] SET QUERY_STORE = ON
GO
ALTER DATABASE [StackOverflow2010] SET QUERY_STORE (OPERATION_MODE = READ_WRITE, DATA_FLUSH_INTERVAL_SECONDS = 300, INTERVAL_LENGTH_MINUTES = 5, MAX_STORAGE_SIZE_MB = 2048, QUERY_CAPTURE_MODE = ALL)
GO


Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.125

Here we create a new schema "change". The code is a bit longer and ensures that an already existing schema will be dropped first with all contained tables and views.

In [19]:
USE [WorkloadAnalysis];
DECLARE @schemaName VARCHAR(50) = 'change';
DECLARE @command    NVARCHAR(200) = NULL;

-- Drop tables and views (if already present)
DECLARE curTruncate CURSOR LOCAL FORWARD_ONLY STATIC READ_ONLY FOR
	SELECT    
		'DROP TABLE ' + QUOTENAME(@schemaName) + '.' + QUOTENAME(t.name) + ';' AS command
	FROM   
				sys.tables t
	INNER JOIN  sys.schemas s ON t.schema_id = s.schema_id
	WHERE
		s.name = @schemaName
	UNION ALL
	SELECT    
		'DROP VIEW ' + QUOTENAME(@schemaName) + '.' + QUOTENAME(t.name) + ';' AS command
	FROM   
				sys.views t
	INNER JOIN  sys.schemas s ON t.schema_id = s.schema_id
	WHERE
		s.name = @schemaName
	;

OPEN curTruncate;
FETCH NEXT FROM curTruncate INTO @command
WHILE @@FETCH_STATUS = 0
BEGIN
    PRINT @command;
    EXECUTE(@command);
    FETCH NEXT FROM curTruncate INTO @command
END
CLOSE curTruncate;
DEALLOCATE curTruncate;

-- Drop an existing schema
SET @command = (
	SELECT	'DROP SCHEMA ' + QUOTENAME(s.name) + ';' AS command 
	FROM	sys.schemas s	
	WHERE   s.name = @schemaName
	);
PRINT @command;
EXECUTE(@command);

-- Create a new schema
SET @command = 'CREATE SCHEMA ' + QUOTENAME(@schemaName);
PRINT @command;
EXECUTE(@command);



/* Kontrollausgaben */
SELECT name FROM sys.databases WHERE name = 'WorkloadAnalysis';
SELECT name FROM WorkloadAnalysis.sys.schemas WHERE name = @schemaName;
SELECT COUNT(*) AS nrOfTables FROM WorkloadAnalysis.sys.tables t INNER JOIN  sys.schemas s ON t.schema_id = s.schema_id AND s.name = @schemaName;

DROP SCHEMA [change];

CREATE SCHEMA [change]

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.134

name WorkloadAnalysis

name change

anzTabellen 0

## Introduce a change before running the second replay
This is just a demo and I am lazy...so we do just that what we probably wouldn't do in reality on a prod system: We just create suggested Missing Indexes and a bit more advanced filtered indexes for our queries.

Hint: The Stack-Overflow-Databases don't have any additional nonclustered indexes (besides one clustered index) per table if you take Brent's backups to set up a vanilla StackOverflow database.


![] (./Bilder/MissingIndexRequest.png "")

In [20]:
USE [StackOverflow2010]
GO
CREATE NONCLUSTERED INDEX posts_idx_OwnerUserId_include
ON [dbo].[Posts] ([OwnerUserId] ASC, [PostTypeId] ASC)
INCLUDE ([ParentId],[Score] )
GO

CREATE NONCLUSTERED INDEX votes_idx_CreationDate_filtered
ON [dbo].[Votes] ([CreationDate] ASC)
WHERE [CreationDate] > '20200512'
GO
CREATE NONCLUSTERED INDEX posts_idx_CreationDate_filtered
ON [dbo].[Posts] ([CreationDate] ASC)
WHERE [CreationDate] > '20200512'
GO

CREATE NONCLUSTERED INDEX votes_idx_VoteTypeId_include
ON [dbo].[Votes] ([VoteTypeId] ASC)
INCLUDE([PostId])
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:04.747

## Configuration for the second Replay
The configuration only has to change slightly...in regard of the schema used by the AnalysisConsumer.
Here's the JSON:
```
{
    "Controller": {

        "Listener":
        {
            "__type": "FileWorkloadListener",
            "Source": "C:\\SQL\\workloadtests\\Demo\\StackOverflow_2010.sqlite",
            "SynchronizationMode": "true"
        },

        "Consumers":
        [
            {
                "__type": "AnalysisConsumer",
                "ConnectionInfo":
            		{
            			"ServerName": ".",
            			"DatabaseName": "WorkloadAnalysis",
                        "SchemaName": "change",
                        "UserName": "sa",
                        "Password": "P4$$w0rd!"
            		},
            		"UploadIntervalSeconds": 60
            },
      	    {
                "__type": "ReplayConsumer",
                "ConnectionInfo":
            		{
            			"ServerName": ".",
                        "DatabaseName": "StackOverflow2010",
                        "UserName": "sa",
                        "Password": "P4$$w0rd!"
            		},
            		"DisplayWorkerStats": "false",
            		"ConsumeResults": "false",
            		"WorkerStatsCommandCount": 1000,
            		"MimicApplicationName": "true",
                	"QueryTimeoutSeconds": 240
            }
        ]
    }
}
```

## Running the second replay
Execute the following command in Powershell (Hint: We use a separate JSON file for the config as it differs from the first replay)
```
& "C:\Program Files\WorkloadTools\SqlWorkload.exe" --file "C:\SQL\workloadtests\Demo\Replay_Change.json" --Log "C:\SQL\workloadtests\Demo\replay$(Get-Date -Format "yyyyMMdd_HHmmss").log"

```

## Clone the database again

In [21]:
DBCC CLONEDATABASE (StackOverflow2010, StackOverflow2010_change);    
GO 

Database cloning for 'StackOverflow2010' has started with target as 'StackOverflow2010_change'.

Database cloning for 'StackOverflow2010' has finished. Cloned database is 'StackOverflow2010_change'.

Database 'StackOverflow2010_change' is a cloned database. This database should be used for diagnostic purposes only and is not supported for use in a production environment.

DBCC execution completed. If DBCC printed error messages, contact your system administrator.

Total execution time: 00:00:03.680